# Import graspi_igraph

Ensure that you are only using python versions higher than 3.7
- Python version can be checked by running *python --version* in the terminal
- it can also be checked on notebook via *!pip --version* in the code section of the notebook

In [2]:
import time

In [3]:
import sys
import os
sys.path.append(os.path.abspath('..'))


# Helper Functions
---


In [4]:
###########################################################################################
# function that makes csv file of dictionary 
###########################################################################################
import csv

def save_dict_to_csv(data_dict, file_name):
    """
    Saves a given dictionary as a CSV file.
    :param data_dict: Dictionary to be saved (keys are filenames, values are nested dictionaries with data)
    :param file_name: Name of the CSV file to be saved (e.g., 'output.csv')
    """
    # Extract headers from the first key
    headers = ["Test File"]  # First column is the file name
    if data_dict:
        sample_key = next(iter(data_dict))
        headers.extend(data_dict[sample_key].keys())
    
    # Write to CSV file
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write headers
        
        for key, values in data_dict.items():
            row = [key] + [values.get(h, None) for h in headers[1:]]
            writer.writerow(row)  # Write data rows
    
    print(f"CSV file '{file_name}' has been successfully saved.")


## Visualization function call
---
change the file name if you want to visualize other result


In [ ]:
!pip install pandas
!pip install matplotlib
import testing_helper as tst


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip



## Testing Code
---
Doing different igraph file test & generating CSV file(for comparing performance)

### Testing :: hwi_igraph_testing_main.py (modified green vertex connection)

In [ ]:
import src.graph as ig
import src.descriptors as ds
import os
import time
import matplotlib.pyplot as plt
import tracemalloc
import src.graph_data_class as gc
import importlib

importlib.reload(ig)  # 강제 리로드
importlib.reload(ds)  # 강제 리로드


current_dir = os.getcwd()
data_path = f"./data/"
descriptors_path = f"../data/descriptors/"
result_path = f"./results/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

for test_file in test_files:
    # g = ig.generateGraph(data_path + test_file + ".txt")
    # test_file = "data_0.5_2.2_001900.txt"
    import time
    tracemalloc.start()
    graph_start = time.time()
    # g,is_2D, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca= ig.generateGraph(test_file)
    g_data = ig.generateGraph(data_path+test_file+".txt")
    # g_data = gc.graph_data_class(g_data.graph, is_2D)
    # g_data.black_vertices = black_vertices
    # g_data.white_vertices = white_vertices
    
    mem_graph = tracemalloc.get_traced_memory()
    graph_end = time.time()
    tracemalloc.stop()
    graph_mem = mem_graph[1]-mem_graph[0]  
    
    desc_start = time.time()
    tracemalloc.start()
    stats = ds.descriptors(g_data, test_file+".txt")
    tracemalloc.stop()
    desc_end = time.time()
    descriptor_time = desc_end - desc_start
    mem_desc = tracemalloc.get_traced_memory()
    descriptor_mem = mem_desc[1]-mem_desc[0]
    #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    with open(descriptors_path + "descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")

    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = graph_end-graph_start
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}




data_0.5_2.2_001900 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_wf_D failed - 0.3209214522690966 is not the same as expected 0.308388
ABS_f_D passed
DISS_wf10_D failed - 0.2166226885998765 is not the same as expected 0.219256
DISS_f10_D failed - 0.4269250756579953 is not the same as expected 0.428209
CT_f_e_conn passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_e_conn passed
CT_e_D_An failed - 1277 is not the same as expected 1278
CT_e_A_Ca failed - 1636 is not the same as expected 1638
CT_f_D_tort1 failed - 0.32920018265481754 is not the same as expected 0.969159
CT_f_A_tort1 failed - 0.2052828809066813 is not the same as expected 0.734028
Total time to calculate graph: 31.284925937652588 second(s)
Total time to calculate descriptors: 14.678931951522827 second(s)
Peak memory usage for graph generation: 79780258 bytes
Peak memory usage for descriptor calculation: 0 bytes
{'STAT_n':

In [ ]:
# change file name 
save_dict_to_csv(time_mem_stats, "performance/stats_new_green_method.csv")